In [5]:
import pandas as pd
from datetime import datetime
from IPython.display import display, HTML

In [6]:
!cp -r ../Scrapers/data .

In [ ]:
!open data/event_data_sherdog.csv
!open data/fighter_info.csv

In [ ]:
# Changing all DQ and Draws to "No Winner"

event_data = pd.read_csv('data/event_data_sherdog.csv')  # Update with your file path

# Update 'winning fighter' to 'No Winner' for draw and disqualification fights
event_data.loc[event_data['winning method'].str.contains('draw|disqualification', case=False, na=False), 
               'winning fighter'] = 'No Winner'

event_data.to_csv('data/event_data_sherdog.csv', index=False)

In [28]:
# Combined W/L/D Record of Fighters Over 40 on Fight Night

event_data = pd.read_csv('data/event_data_sherdog.csv')

# Filter for fighters who were 40 or older on fight night
fighters_40_or_older = event_data[(event_data['fighter1_age_on_fight_night'] >= 40) | 
                                  (event_data['fighter2_age_on_fight_night'] >= 40)]

# Function to categorize the method as 'KO', 'Submission', 'Decision', or 'Other'
def categorize_method(method):
    method_lower = method.lower()
    if 'ko' in method_lower or 'submission' in method_lower or 'decision' in method_lower:
        return method
    elif 'draw' in method_lower or 'disqualification' in method_lower:
        return 'Other'
    else:
        return method

# Initialize dictionaries to track combined win/loss/draw records
combined_record = {'W': 0, 'L': 0, 'D': 0}

# Iterate over each row in the filtered dataset
for _, row in fighters_40_or_older.iterrows():
    method = categorize_method(row['winning method'])
    
    if method == 'Other':
        for fighter, age_col in [('fighter 1', 'fighter1_age_on_fight_night'), ('fighter 2', 'fighter2_age_on_fight_night')]:
            if row[age_col] >= 40:
                if 'draw' in method.lower():
                    combined_record['D'] += 1
                else:
                    combined_record['L'] += 1
    else:
        for fighter, age_col in [('fighter 1', 'fighter1_age_on_fight_night'), ('fighter 2', 'fighter2_age_on_fight_night')]:
            if row[age_col] >= 40:
                if row['winning fighter'].lower() == row[fighter].lower():
                    combined_record['W'] += 1
                else:
                    combined_record['L'] += 1

# Print the combined win/loss/draw records
print(f"Combined Record for Fighters Over 40: {combined_record['W']} Wins, {combined_record['L']} Losses, {combined_record['D']} Draws")


Combined Record for Fighters Over 40: 72 Wins, 133 Losses, 0 Draws


In [30]:
# Winning and Losing Methods for Fighters Over 40 on Fight Night

event_data = pd.read_csv('data/event_data_sherdog.csv')

# Filter for fighters who were 40 or older on fight night
fighters_40_or_older = event_data[(event_data['fighter1_age_on_fight_night'] >= 40) | 
                                  (event_data['fighter2_age_on_fight_night'] >= 40)]

# Function to categorize the method as 'KO', 'Submission', or 'Other'
def categorize_method(method):
    method_lower = method.lower()
    if 'ko' in method_lower or 'submission' in method_lower:
        return method
    elif 'draw' in method_lower or 'disqualification' in method_lower:
        return 'Other'
    else:
        return method

# Initialize lists to hold win, loss, and other methods
win_methods = []
loss_methods = []
other_methods = []
other_fights = []

# Initialize dictionaries to track win/loss/draw records
records = {}

# Iterate over each row in the filtered dataset
for _, row in fighters_40_or_older.iterrows():
    method = categorize_method(row['winning method'])
    
    if method == 'Other':
        other_methods.append(row['winning method'])
        other_fights.append(row)  # Store the entire row if it's an "Other" method
    else:
        if row['fighter1_age_on_fight_night'] >= 40:
            if row['winning fighter'].lower() == row['fighter 1'].lower():
                win_methods.append(row['winning method'])
            else:
                loss_methods.append(row['winning method'])
        
        if row['fighter2_age_on_fight_night'] >= 40:
            if row['winning fighter'].lower() == row['fighter 2'].lower():
                win_methods.append(row['winning method'])
            else:
                loss_methods.append(row['winning method'])

# Convert the lists into DataFrames for easier analysis
win_methods_df = pd.DataFrame(win_methods, columns=['Winning Method'])
loss_methods_df = pd.DataFrame(loss_methods, columns=['Losing Method'])
other_methods_df = pd.DataFrame(other_methods, columns=['Other Method'])

# Convert other_fights to a DataFrame
other_fights_df = pd.DataFrame(other_fights)

# Count the occurrences of each winning and losing method
win_method_counts = win_methods_df['Winning Method'].value_counts()
loss_method_counts = loss_methods_df['Losing Method'].value_counts()
other_method_counts = other_methods_df['Other Method'].value_counts()

# Display the results
print("Winning Methods:")
print(win_method_counts)
print("\nLosing Methods:")
print(loss_method_counts)
print("\nOther Methods:")
print(other_method_counts)

# Print all the fights that involved "Other" methods
print("\nFights with Other Methods:")
display(other_fights_df)


Winning Methods:
Winning Method
decision (unanimous)                       19
tko (punches)                               9
submission (rear-naked choke)               8
decision (split)                            6
ko (punches)                                6
ko (punch)                                  5
submission (armbar)                         3
tko (elbows and punches)                    2
submission (neck crank)                     1
ko (knee)                                   1
submission (anaconda choke)                 1
submission (arm-triangle choke)             1
tko (elbows and knees)                      1
ko (elbow)                                  1
technical submission (rear-naked choke)     1
submission (ezekiel choke)                  1
ko (knee to the body)                       1
decision (majority)                         1
submission (face crank)                     1
tko (punch)                                 1
submission (scarf hold)                     1
tk

,event name,event location,event date,fighter 1,fighter 2,fighter 1 id,fighter 2 id,weight class,winning fighter,winning method,...,fighter1_current_layoff,fighter2_current_layoff,fighter1_height_in_inches,fighter2_height_in_inches,fighter1_total_wins,fighter1_total_losses,fighter2_total_wins,fighter2_total_losses,fighter1_age_on_fight_night,fighter2_age_on_fight_night
2740,ufc 222 - cyborg vs. kunitskaya,"t-mobile arena, las vegas, nevada, united states",2018-03-03 08:00:00,cb dollaway,hector lombard,22350,11292,middleweight,c.b. dollaway,disqualification (punches after the bell),...,2170.0,2163.0,74.0,69.0,17,10,34.0,11.0,34,40
